# BertEncoder 성능 평가

In [2]:
from bert_feature.bert_encoder import BertFeatureExtractor
from tqdm import tqdm

/home/eden/anaconda3/envs/nlp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
import json

class G2PDataset(Dataset):
    def __init__(self, json_data, tokenizer, max_length=256):
        self.data = json_data["data"]
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        g2p_candidates = item["g2p_result"]
        target_sentence = item["sentence"]  # sentence is a list with 1 string


        return {
            
        }

def get_dataloader(json_path, tokenizer, batch_size=8, shuffle=True):
    with open(json_path, "r", encoding="utf-8") as f:
        json_data = json.load(f)
    dataset = G2PDataset(json_data, tokenizer)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)


In [4]:
bert_encoder = BertFeatureExtractor()

Using device: cuda


In [5]:
dataloader = get_dataloader(
    '/home/eden/Documents/JNU/2025-1/Natural-Language-Processing/Natural-Language-Processing-Project/data/001.문서요약/1.Training_1216_add/Train_법률_data/train_original_preprocessed.json',
    bert_encoder.tokenizer,
    batch_size=1,
    shuffle=True,
    
)

In [6]:
json_path = '/home/eden/Documents/JNU/2025-1/Natural-Language-Processing/Natural-Language-Processing-Project/data/001.문서요약/1.Training_1216_add/Train_법률_data/train_original_preprocessed.json'
with open(json_path, "r", encoding="utf-8") as f:
        json_data = json.load(f)

In [13]:
all_similarities = []
to_get_similarities = json_data["data"][:1000]  # Limit to 1000 samples for testing
for data in tqdm(to_get_similarities):
    g2p_candidates = data["g2p_result"]
    target_sentence = data["sentence"]  # sentence is a list with 1 string
    g2p_mean_embedding = bert_encoder.extract_features(g2p_candidates, pooling_strategy='max')
    target_mean_embedding = bert_encoder.extract_features(target_sentence, pooling_strategy='max')

    similarities = torch.tensor([torch.cosine_similarity(g2p_mean_embedding, target_mean_embedding).item() for g2p_mean_embedding in g2p_mean_embedding])
    all_similarities.append(similarities.mean())
    


100%|██████████| 1000/1000 [00:09<00:00, 109.95it/s]


In [14]:
all_similarities = torch.stack(all_similarities)
print(all_similarities.shape)
print(all_similarities.mean())


torch.Size([1000])
tensor(0.9621)


# FusionEmbedding 성능 평가

In [16]:
for batch in dataloader:
    print(batch)
    g2p_mean_embedding = bert_encoder.extract_features(batch, pooling_strategy="mean")


    similarities = [torch.cosine_similarity(bert_encoder[0], bert_encoder[i]) for i in range(1, len(batch["g2p_results"]))]


[('타인의 등록상표와 동일 또는 유사한 표장을 이용한 경우라고 하더라도 그것이 상표의 본질적인 기능이라고 할 수 있는 출처표시를 위한 것이 아니라 순전히 디자인적으로만 사용되는 등으로 상표의 사용으로 인식될 수 없는 경우에는 등록상표의 상표권을 침해한 행위로 볼 수 없고,',), ('타인의 등록쌍표와 동일 또는 유사한 표장을 이용한 경우라고 하더라도 그것이 상표의 본질적인 기능이라고 할 쑤 있는 출처표시를 위한 것이 아니라 순전히 디자인적으로만 사용되는 등으로 상표의 사용으로 인식될 쑤 없는 경우에는 등록쌍표의 상표권을 침해한 행위로 볼 쑤 없고,',), ('타이늬 등록상표와 동일 또는 유사한 표장을 이용한 경우라고 하더라도 그거시 상표의 본질저긴 기능이라고 할 쑤 인는 출처표시를 위한 거시 아니라 순전히 디자인저그로만 사용되는 등으로 상표의 사용으로 인식될 쑤 없는 경우에는 등록상표의 상표궈늘 침해한 행위로 볼 쑤 없고,',), ('타이늬 등록쌍표와 동일 또느 뉴사한 표장으 리용한 경우라고 하더라도 그거시 상표의 본질저긴 기능이라고 할 쑤 있는 출처표시르 뤼한 거시 아니라 순전히 디자인저그로만 사용되는 등으로 상표의 사용으로 인식될 쑤 없는 경우에는 등록쌍표의 상표궈늘 침해한 행위로 볼 쑤 없고,',), ('타인의 등록쌍표와 동일 또느 뉴사한 표장으 리용한 경우라고 하더라도 그것이 상표의 본질적인 기능이라고 할 쑤 있는 출처표시르 뤼한 것이 아니라 순전히 디자인적으로만 사용되는 등으로 상표의 사용으로 인식될 쑤 없는 경우에는 등록쌍표의 상표권을 침해한 행위로 볼 쑤 없고,',), ('타이늬 등록상표와 동일 또느 뉴사한 표장으 리용한 경우라고 하더라도 그거시 상표의 본질저긴 기능이라고 할 수 있는 출처표시르 뤼한 거시 아니라 순전히 디자인저그로만 사용되는 등으로 상표의 사용으로 인식뙬 수 없는 경우에는 등록상표의 상표궈늘 침해한 행위로 볼 수 없고,',)]


ValueError: not enough values to unpack (expected 2, got 1)